In [1]:
pip install transformers datasets peft

  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
   ---------------------------------------- 0.0/480.6 kB ? eta -:--:--
   --------- ------------------------------ 112.6/480.6 kB 3.2 MB/s eta 0:00:01
   ---------------------------------------  471.0/480.6 kB 5.8 MB/s eta 0:00:01
   ---------------------------------------- 480.6/480.6 kB 5.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/320.7 kB ? eta -:--:--
   --------------------------------------  317.4/320.7 kB 19.2 MB/s eta 0:00:01
   ---------------------------------------- 320.7/320.7 kB 9.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/333.2 kB ? eta -:--:--
   --------------------------------------- 333.2/333.2 kB 10.4 MB/s eta 0:00:00
Using cached dill-0.3.8-py3-none-any.whl (116 kB)
   ---------------------------------------- 0.0/143.5 kB ? eta -:--:--
   ---------------------------------------- 143.5/143.5 kB 8.9 MB/s eta 0:00:00
  Attempting uninstall: dill
    Found existin


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\ss1516\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
import json
from datasets import Dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
import torch
from peft import get_peft_model, LoraConfig

# 1. Load the distilgpt2 model and tokenizer
# tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2')
# model = GPT2LMHeadModel.from_pretrained('distilgpt2')
# # Set the pad_token to eos_token
# tokenizer.pad_token = tokenizer.eos_token
# model.config.pad_token_id = tokenizer.eos_token_id


tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.add_special_tokens({'eos_token': '<|endoftext|>', 'pad_token': '<|endoftext|>'})

model = GPT2LMHeadModel.from_pretrained('gpt2')
model.resize_token_embeddings(len(tokenizer))
model.config.eos_token_id = tokenizer.eos_token_id
model.config.pad_token_id = tokenizer.pad_token_id

print("model loaded")

# 2. Apply LoRA
config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["c_attn", "c_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)
print("LoRA applied")
model.print_trainable_parameters()
print("trainable parameters printed")

# 3. Load and format your dataset (same as before)
# ... (use the same load_dataset function as earlier)
# def load_dataset(file_path):
#     with open(file_path, 'r') as file:
#         data = json.load(file)
    
#     # Limit to 1000 examples
#     data = data[:100]
    
#     formatted_data = []
#     for item in data:
#         instruction = item.get('instruction', '')
#         input_text = item.get('input', '')
#         output_text = item.get('output', '')
        
#         # Construct the prompt
#         if input_text:
#             prompt = f"{instruction}\n\nInput: {input_text}\n\nResponse:"
#         else:
#             prompt = f"{instruction}\n\nResponse:"
        
#         # Combine prompt and output
#         full_text = prompt + " " + output_text
        
#         formatted_data.append({'text': full_text})
    
#     return Dataset.from_dict({'text': [item['text'] for item in formatted_data]})
def load_dataset(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    
   
    data = data[:1000] #2k 
    
    formatted_data = []
    for item in data:
        instruction = "You are an experienced medical doctor specializing in mental health. Provide a detailed and compassionate response to the patient's description." #item.get('instruction', '')
        input_text = item.get('input', '')
        output_text = item.get('output', '')
        
        # Construct the prompt
        if input_text:
            prompt = f"{instruction}\n\nPatient's Description: {input_text}\n\nDoctor's Response"
        else:
            prompt = f"{instruction}\n\nDoctor's Response"
        
        formatted_data.append({
            'text': prompt,
            'label': output_text
        })
    
    return Dataset.from_dict({
        'text': [item['text'] for item in formatted_data],
        'label': [item['label'] for item in formatted_data]
    })


dataset = load_dataset('chatdoctor5k.json')  #'chatdoctor5k.json')

# 4. Tokenize the dataset
# def tokenize_function(examples):
#     return tokenizer(examples['text'], truncation=True, padding="longest", max_length=128 )

# tokenized_dataset = dataset.map(tokenize_function, batched=True)
def tokenize_function(examples):
    inputs = tokenizer(
        examples['text'],
        truncation=True,
        max_length=128,
        padding='longest'
    )
    outputs = tokenizer(
        examples['label'],
        truncation=True,
        max_length=128,
        padding='longest'
    )
    
    # Concatenate the inputs and outputs
    input_ids = []
    labels = []
    attention_masks = []
    for i in range(len(inputs['input_ids'])):
        input_id = inputs['input_ids'][i] + outputs['input_ids'][i]
        attention_mask = inputs['attention_mask'][i] + outputs['attention_mask'][i]
        label = [-100] * len(inputs['input_ids'][i]) + outputs['input_ids'][i]
        
        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        labels.append(label)
    
    return {
        'input_ids': input_ids,
        'attention_mask': attention_masks,
        'labels': labels
    }

# tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=dataset.column_names  # Remove original columns
)

# 5. Prepare data collator
from transformers import default_data_collator

data_collator = default_data_collator

# data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# 6. Set training arguments
training_args = TrainingArguments(
    output_dir='./lora_results',
    overwrite_output_dir=True,
    learning_rate=3e-4,
    per_device_train_batch_size=4, #4 or 8
    num_train_epochs=3, #3 or more
    logging_steps=10,
    save_steps=50,
    save_total_limit=2,
    remove_unused_columns=True,
    fp16=torch.cuda.is_available(),
    gradient_accumulation_steps=1,
    # evaluation_strategy="steps",
    # eval_steps=200,
)

# 7. Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset,
)

# 8. Train the model
trainer.train()

# 9. Save the LoRA adapter
model.save_pretrained('./trained_lora_2k')
tokenizer.save_pretrained('./trained_lora_2k')


: 

In [50]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

# Load the fine-tuned model and tokenizer
model = GPT2LMHeadModel.from_pretrained('./trained_lora')
tokenizer = GPT2Tokenizer.from_pretrained('./trained_lora')

model.to('cuda' if torch.cuda.is_available() else 'cpu')
model.eval()

# Prepare a prompt
instruction = "You are an experienced medical doctor specializing in mental health. Provide a detailed and compassionate response to the patient's description."
input_text = "I have been experiencing sudden and frequent panic attacks."
prompt = f"{instruction}\n\nPatient's Description: {input_text}\n\nDoctor's Response:"

inputs = tokenizer(prompt, return_tensors='pt').to(model.device)

with torch.no_grad():
    # outputs = model.generate(
    #     **inputs,
    #     max_new_tokens=100,
    #     do_sample=True,
    #     top_p=0.9,
    #     temperature=0.75,
    # )
    outputs = model.generate(
    **inputs,
    max_new_tokens=200,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    repetition_penalty=1.1,
    no_repeat_ngram_size=3,
)


generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


You are an experienced medical doctor specializing in mental health. Provide a detailed and compassionate response to the patient's description.

Patient's Description: I have been experiencing sudden and frequent panic attacks.

Doctor's Response: It is normal for you, especially if your symptoms aren't completely consistent with what we've described here. However there may be some other conditions that might indicate something more serious than sleep apnea or dizziness. We'll need blood tests to confirm those factors (including any abnormal oxygenation levels), neurological imaging procedures to check your breathing, psychological evaluation of possible medications used to treat this condition such as medication changes, etc., so that we can diagnose it correctly.


In [1]:
list = [1,2,3,4,5,6]
list[-3:]

[4, 5, 6]

In [4]:
import numpy as np
np.mean(list)

3.5

In [5]:
data= """Generated Response: Based on your symptoms, we need to perform a series of medical tests to confirm the diagnosis. We will need to conduct a complete physical skin exam, hematologic tests, hematology tests, heme (hematologic test), and hematologic test. We may also need to conduct other diagnostic procedures such as hematologic testing, hematological tests, heparin examination, hematocrit test, hematochemical test, heme replacement, and kidney function test. Additionally, we may need to do a urinalysis to check for any kidney damage.
Perplexity: 3.62
prompt: {'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': "Doctor, I've been experiencing a lack of growth, delusions or hallucinations, and fainting. What could be causing these symptoms?", 'output': "The symptoms you described could be caused by a number of things, but based on what you've told me, it's possible that you have Asperger syndrome."} and perplexity score: 3.6189939975738525
{'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': 'Doctor, I have been experiencing some symptoms lately. I have been having obsessions and compulsions, along with temper problems. Also, I faint sometimes.', 'output': 'Based on what you are telling me, it seems like you might have Asperger syndrome.'}
Welcome to the Medical Chatbot. Type 'exit' or 'quit' to end the conversation.

Generated Response: Okay, it sounds like you may have a severe allergic reaction to mifepristone. To confirm this, we will need to perform a complete blood count and conduct a biopsy. We will also need to conduct a urinalysis to check for any abnormalities in your blood. Additionally, we will conduct a complete physical skin exam to determine if you have any allergies or sensitivities. Lastly, we will run a complete blood culture to check for IgE antibodies and other diagnostic procedures to diagnose any other health issues that may be contributing to your condition.
Perplexity: 3.59
prompt: {'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': 'Doctor, I have been experiencing some symptoms lately. I have been having obsessions and compulsions, along with temper problems. Also, I faint sometimes.', 'output': 'Based on what you are telling me, it seems like you might have Asperger syndrome.'} and perplexity score: 3.5851686000823975
{'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': 'Doctor, I??e been experiencing excessive anger, lack of growth, antisocial behavior, obsessions and compulsions, and even delusions or hallucinations. What could be causing these symptoms?', 'output': 'It sounds like you may be experiencing Asperger syndrome, which is a developmental disorder characterized by difficulties with social interaction and communication, as well as obsessive interests and repetitive behaviors. It can also sometimes involve anger and even delusions or hallucinations. We can run some tests and discuss treatment options to help manage your symptoms.'}
Welcome to the Medical Chatbot. Type 'exit' or 'quit' to end the conversation.

Generated Response: Based on your symptoms, it is possible that you may have bipolar disorder. To confirm this, we will need to conduct a complete physical skin exam, hematologic tests, and a lipid panel to determine the level of testosterone in your blood. Additionally, we will also need to conduct other diagnostic procedures such as a complete blood count (CBC), hematologic test, and lipid panel to check for any abnormalities in your liver. Additionally, a complete blood culture (CBC) will be necessary to rule out any other disorders.
Perplexity: 2.92
prompt: {'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': 'Doctor, I??e been experiencing excessive anger, lack of growth, antisocial behavior, obsessions and compulsions, and even delusions or hallucinations. What could be causing these symptoms?', 'output': 'It sounds like you may be experiencing Asperger syndrome, which is a developmental disorder characterized by difficulties with social interaction and communication, as well as obsessive interests and repetitive behaviors. It can also sometimes involve anger and even delusions or hallucinations. We can run some tests and discuss treatment options to help manage your symptoms.'} and perplexity score: 2.9243052005767822
{'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': 'Doctor, I have been having nightmares, anxiety and nervousness, temper problems, and have fainted a few times. What could be wrong?', 'output': 'It sounds like you may have Asperger syndrome. Have you experienced any difficulties with communication, social interaction or repetitive behaviors? It is important to consult with a specialist for further evaluation and treatment options.'}
Welcome to the Medical Chatbot. Type 'exit' or 'quit' to end the conversation.

Generated Response: Based on your symptoms, it is possible that you may have a hypothyroidism. To confirm this, we will need to perform a complete physical skin exam, hematologic tests, and a urinalysis. We will also need to conduct a plain x-ray of your body to check for any abnormalities in your thyroid. Additionally, we will also need a complete blood count (CBC) to check for abnormalities in your kidneys. Lastly, we will conduct a radiographic imaging procedure to see if any of your kidneys are functioning properly.
Perplexity: 2.79
prompt: {'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': 'Doctor, I have been having nightmares, anxiety and nervousness, temper problems, and have fainted a few times. What could be wrong?', 'output': 'It sounds like you may have Asperger syndrome. Have you experienced any difficulties with communication, social interaction or repetitive behaviors? It is important to consult with a specialist for further evaluation and treatment options.'} and perplexity score: 2.7933542728424072
{'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': "Doctor, I've been having some strange symptoms lately. I've been acting really antisocial and hostile. Also, I've been feeling really down and sometimes even have psychotic symptoms. What could be wrong with me?", 'output': "Based on your symptoms, it's possible that you may be experiencing Asperger syndrome. We will need to conduct some tests and assessments to confirm the diagnosis."}
Welcome to the Medical Chatbot. Type 'exit' or 'quit' to end the conversation.

Generated Response: Based on your symptoms, it seems like you may be suffering from schizophrenia. To confirm this, we need to perform a series of tests. We will start with a complete blood count (CBC) and a complete physical skin exam (CBC). We will also need to conduct a urinalysis to check for any abnormal fluid levels. Additionally, we will need to do a biopsy and other diagnostic procedures such as an interview, evaluation, and consultation. Finally, we will conduct a complete physical physical skin exam to check for signs of mental illness.
Perplexity: 2.86
prompt: {'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': "Doctor, I've been having some strange symptoms lately. I've been acting really antisocial and hostile. Also, I've been feeling really down and sometimes even have psychotic symptoms. What could be wrong with me?", 'output': "Based on your symptoms, it's possible that you may be experiencing Asperger syndrome. We will need to conduct some tests and assessments to confirm the diagnosis."} and perplexity score: 2.860464096069336
{'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': 'Doctor, I have been experiencing severe pain in my tongue, and I also have a fever. What could be the issue?', 'output': 'Based on your symptoms, it appears that you may have mucositis, which is inflammation of the mucous membrane lining the digestive tract.'}
Welcome to the Medical Chatbot. Type 'exit' or 'quit' to end the conversation.

Generated Response: In order to diagnose the condition, we will need to perform several medical tests including a complete blood count, hematologic tests, hematology tests, and hematologic examination. We will also need to conduct other diagnostic procedures such as hematologic test, hematocrit tests, and lipid panel. Additionally, we may need to do an interview to determine your health status.
Perplexity: 3.21
prompt: {'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': 'Doctor, I have been experiencing severe pain in my tongue, and I also have a fever. What could be the issue?', 'output': 'Based on your symptoms, it appears that you may have mucositis, which is inflammation of the mucous membrane lining the digestive tract.'} and perplexity score: 3.2122554779052734
{'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': 'Doctor, I am having difficulty in swallowing, what could be the reason for it?', 'output': 'It could be due to mucositis.'}
Welcome to the Medical Chatbot. Type 'exit' or 'quit' to end the conversation.

Generated Response: Based on your symptoms, we will need to perform a series of medical tests to confirm your diagnosis. We will need to conduct a complete blood count (CBC) and glucose measurement (Glucose measurement), a complete blood panel (CBC), and a lipid panel (Lancet panel) to check your cholesterol levels. Additionally, we will also need to conduct other diagnostic procedures such as a urinalysis and evaluation of your bowel function. Additionally, an electrolytes panel (Abdominal fluid replacement) and a plain x-ray may be necessary to rule out any possible causes of your condition.
Perplexity: 3.03
prompt: {'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': 'Doctor, I am having difficulty in swallowing, what could be the reason for it?', 'output': 'It could be due to mucositis.'} and perplexity score: 3.0332791805267334
{'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': 'Hi doctor, I have a swollen tongue, tongue pain, and mouth pain. What could be causing these symptoms?', 'output': 'Based on your symptoms, you may be experiencing mucositis.'}
Welcome to the Medical Chatbot. Type 'exit' or 'quit' to end the conversation.

Generated Response: Based on your symptoms, it is possible that you may have a yeast infection. To confirm this, we will need to perform a complete physical skin exam and evaluation. We will also need to conduct a radiographic imaging procedure, such as a plain x-ray, to check for any abnormalities in your body. Additionally, we may need to do a urinalysis and a complete physical culture of your bladder. Additionally, a plain x image of your penis and rectum will help us determine if there is any inflammation or infection.
Perplexity: 3.21
prompt: {'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': 'Hi doctor, I have a swollen tongue, tongue pain, and mouth pain. What could be causing these symptoms?', 'output': 'Based on your symptoms, you may be experiencing mucositis.'} and perplexity score: 3.212578058242798
{'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': "Doctor, my tongue is swollen and it's really bothering me. What could be the reason for this?", 'output': 'Based on your symptoms, it is possible that you are experiencing mucositis. This is a condition where the lining of your mouth becomes inflamed and swollen, often as a result of chemotherapy or radiation therapy.'}
Welcome to the Medical Chatbot. Type 'exit' or 'quit' to end the conversation.

Generated Response: Based on your symptoms, we need to perform a few tests to determine if you have a tongue ulcer. We will start with a complete physical skin exam, hematologic tests, hematology tests, heme panel, hematological tests, and hematologic examination. We will also need to do a urinalysis to check for any fluid buildup in your bladder. Additionally, we may need to conduct a urinal culture to check for other infections. Finally, we may also need to consider a biopsy to confirm the diagnosis.
Perplexity: 4.04
prompt: {'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': "Doctor, my tongue is swollen and it's really bothering me. What could be the reason for this?", 'output': 'Based on your symptoms, it is possible that you are experiencing mucositis. This is a condition where the lining of your mouth becomes inflamed and swollen, often as a result of chemotherapy or radiation therapy.'} and perplexity score: 4.03501558303833
{'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': 'Doctor, I have been having fever, tongue lesions, and gum pain. What might be my problem?', 'output': 'Well, based on your symptoms, you might be having mucositis.'}
Welcome to the Medical Chatbot. Type 'exit' or 'quit' to end the conversation.

Generated Response: We will need to perform some medical tests to confirm your diagnosis. We will need a Hematologic test, hematologic test, and a Hemoglobin A1c measurement. Additionally, we may need to conduct a Hemoglobin B1c measurement to check for any abnormalities in your blood. Finally, we may also need to conduct an Electrocardiogram (Egocentrism) to check for heart disease.
Perplexity: 3.14
prompt: {'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': 'Doctor, I have been having fever, tongue lesions, and gum pain. What might be my problem?', 'output': 'Well, based on your symptoms, you might be having mucositis.'} and perplexity score: 3.143325090408325
{'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': 'Hi doctor, I have irregular appearing nails, hand and finger pain. The skin on my arm or hand also looks infected and I am experiencing sinus congestion. What could this be?', 'output': "It sounds like you may have paronychia. It's an infection that occurs around the nails and can cause pain and swelling in the affected area. The sinus congestion is likely not related and may be a separate issue. We can prescribe antibiotics to help clear up the infection."}
Welcome to the Medical Chatbot. Type 'exit' or 'quit' to end the conversation.

Generated Response: In order to confirm your diagnosis, we will need to perform some medical tests. We will need to conduct a radiographic imaging procedure to check for any signs of infection. We will also need to conduct other diagnostic procedures such as an interview, evaluation, and consultation. Additionally, we may need to conduct an ophthalmologic examination and evaluation to determine if there is any underlying condition that warrants further medical examination. Additionally, a corneal transplant may be necessary to treat the affected area.
Perplexity: 2.96
prompt: {'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': 'Hi doctor, I have irregular appearing nails, hand and finger pain. The skin on my arm or hand also looks infected and I am experiencing sinus congestion. What could this be?', 'output': "It sounds like you may have paronychia. It's an infection that occurs around the nails and can cause pain and swelling in the affected area. The sinus congestion is likely not related and may be a separate issue. We can prescribe antibiotics to help clear up the infection."} and perplexity score: 2.963890314102173
{'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': "Doctor, my foot is swollen and it's painful. I don't know what happened but I can barely walk on it.", 'output': "Based on your description, it sounds like you may have paronychia. It's an infection that occurs around the nail bed, often affecting the toes or fingers. Have you noticed any redness or tenderness around your toenail?"}
Welcome to the Medical Chatbot. Type 'exit' or 'quit' to end the conversation.

Generated Response: Based on your symptoms, it sounds like you may have a fibromyalgia. To confirm this, we will need to perform a few medical tests. We will need to start with a plain x-ray, hematologic test, hematology tests, hematological tests, heme (hematologic test), urinalysis, and a complete blood count. Additionally, we may need to conduct a complete blood measurement of your heart. In addition, we may also need to conduct other diagnostic procedures such as a complete blood work, kidney function tests, and kidney function tests.
Perplexity: 3.53
prompt: {'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': "Doctor, my foot is swollen and it's painful. I don't know what happened but I can barely walk on it.", 'output': "Based on your description, it sounds like you may have paronychia. It's an infection that occurs around the nail bed, often affecting the toes or fingers. Have you noticed any redness or tenderness around your toenail?"} and perplexity score: 3.5292959213256836
{'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': 'I have skin dryness, peeling, scaliness or roughness, and the skin on my leg or foot looks infected. Also, my skin appears abnormal and I am experiencing sinus congestion. What could be the problem, doctor?', 'output': 'Based on the symptoms, you most likely have Paronychia - an infection that affects the skin around your fingernail or toenail. However, sinus congestion is not typically associated with this disease. I recommend you to take antibiotic or antifungal medications depending on the severity of the infection.'}
Welcome to the Medical Chatbot. Type 'exit' or 'quit' to end the conversation.

Generated Response: Based on your symptoms, it is possible that you may have a rare form of chlamydia. We will need to conduct some medical tests to confirm the diagnosis. We will also need to do a complete physical skin exam, urinalysis, and evaluation of your skin. Additionally, we may need to perform a urinalysis to check for any infection.
Perplexity: 3.16
prompt: {'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': 'I have skin dryness, peeling, scaliness or roughness, and the skin on my leg or foot looks infected. Also, my skin appears abnormal and I am experiencing sinus congestion. What could be the problem, doctor?', 'output': 'Based on the symptoms, you most likely have Paronychia - an infection that affects the skin around your fingernail or toenail. However, sinus congestion is not typically associated with this disease. I recommend you to take antibiotic or antifungal medications depending on the severity of the infection.'} and perplexity score: 3.15982723236084
{'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': "Doctor, my foot is swollen and it hurts. I don't know what's wrong.", 'output': "Based on what you're telling me, it sounds like you may have paronychia. It is an infection around the nail bed that can cause swelling and pain in the affected area."}
Welcome to the Medical Chatbot. Type 'exit' or 'quit' to end the conversation.

Generated Response: Okay, you may need to undergo some medical tests to determine the extent of your swelling. We'll need to perform a complete physical skin exam, hematologic test, hematology, urinalysis, hematological test, hemostasis, hematocrit measurement, hematochemistry, hematolog test, and kidney function tests. Additionally, we may need to conduct a radiographic imaging procedure to check for any abnormalities in your blood vessels.
Perplexity: 4.00
prompt: {'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': "Doctor, my foot is swollen and it hurts. I don't know what's wrong.", 'output': "Based on what you're telling me, it sounds like you may have paronychia. It is an infection around the nail bed that can cause swelling and pain in the affected area."} and perplexity score: 4.004613399505615
{'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': 'Doctor, I have been experiencing foot and hand pain recently. In addition, my toes and fingers have a noticeable swelling. What could be causing these symptoms?', 'output': 'Based on your symptoms, it sounds like you may have Paronychia which is an infection that affects the skin surrounding a toenail or fingernail.'}
Welcome to the Medical Chatbot. Type 'exit' or 'quit' to end the conversation.

Generated Response: We will need to perform some medical tests to confirm your diagnosis. We will need a plain x-ray to check for any fractures or other injuries. We will also need to conduct a complete physical skin exam to rule out any other underlying causes. Additionally, we may need to do a radiographic imaging procedure to check for abnormalities in your feet. Finally, we may require a plain xelvis examination to check for other conditions that might be contributing to the pain.
Perplexity: 4.21
prompt: {'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': 'Doctor, I have been experiencing foot and hand pain recently. In addition, my toes and fingers have a noticeable swelling. What could be causing these symptoms?', 'output': 'Based on your symptoms, it sounds like you may have Paronychia which is an infection that affects the skin surrounding a toenail or fingernail.'} and perplexity score: 4.20528507232666
{'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': 'Doctor, I have a big swollen lip and I have an open wound in my jaw. What could be causing this?', 'output': 'Based on your symptoms, it seems like you have an open wound of the jaw which may have resulted in the lip swelling.'}
Welcome to the Medical Chatbot. Type 'exit' or 'quit' to end the conversation.

Generated Response: Based on your symptoms, it is possible that you may have a ruptured lung. We will need to perform some radiographic imaging procedures to confirm the diagnosis. We will also need to do a complete physical skin exam, hematologic tests, hematology tests, and hematologic test, and hemostasis tests. Additionally, we may need to conduct other diagnostic procedures such as an interview, evaluation, and consultation.
Perplexity: 3.44
prompt: {'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': 'Doctor, I have a big swollen lip and I have an open wound in my jaw. What could be causing this?', 'output': 'Based on your symptoms, it seems like you have an open wound of the jaw which may have resulted in the lip swelling.'} and perplexity score: 3.44016695022583
{'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': 'Doctor, I have been experiencing mouth pain and facial pain with some symptoms on my face. What can it be?', 'output': 'It seems that you might have an open wound on your jaw which is causing the pain and other symptoms.'}
Welcome to the Medical Chatbot. Type 'exit' or 'quit' to end the conversation.

Generated Response: Based on your symptoms, we need to perform a radiographic imaging procedure to determine if you have a flat nose or flat mouth. We will also need to do a plain x-ray of your face to check for any signs of infection. Additionally, we may need to conduct a radiographic computed tomography (CT scan) to check for other possible causes of the condition.
Perplexity: 3.41
prompt: {'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': 'Doctor, I have been experiencing mouth pain and facial pain with some symptoms on my face. What can it be?', 'output': 'It seems that you might have an open wound on your jaw which is causing the pain and other symptoms.'} and perplexity score: 3.408458709716797
{'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': "Doctor, I have a wound on my jaw and my lip is swelling. I'm also having a lot of mouth pain and some strange symptoms on my face. What could be wrong with me?", 'output': 'Based on your symptoms, it sounds like you may have an open wound on your jaw that has become infected. This can cause swelling of the lip and pain in the mouth, as well as other symptoms on the face. We need to clean the wound and possibly prescribe antibiotics to help clear the infection.'}
Welcome to the Medical Chatbot. Type 'exit' or 'quit' to end the conversation.

Generated Response: Based on your symptoms, it seems like you may have a type 2 diabetes mellitus. To confirm the diagnosis, we need to perform a radiographic imaging procedure to check for any signs of inflammation. We will also need to do a complete physical skin exam to check for signs of inflammation and other complications. Additionally, we may need to conduct a urinalysis to check for kidney function.      
Perplexity: 3.34
prompt: {'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': "Doctor, I have a wound on my jaw and my lip is swelling. I'm also having a lot of mouth pain and some strange symptoms on my face. What could be wrong with me?", 'output': 'Based on your symptoms, it sounds like you may have an open wound on your jaw that has become infected. This can cause swelling of the lip and pain in the mouth, as well as other symptoms on the face. We need to clean the wound and possibly prescribe antibiotics to help clear the infection.'} and perplexity score: 3.3367278575897217
{'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': "Hi doctor, I have a wound on my jaw that won't heal and my wrist is swelling.", 'output': "Based on your symptoms you might have an infection called Ludwig's angina, which can spread from the jaw to the neck and chest. It can also cause swelling in your wrist if left untreated."}
Welcome to the Medical Chatbot. Type 'exit' or 'quit' to end the conversation.

Generated Response: Okay, we need to perform some medical tests to determine if you have any fractures. We will need to do a radiographic imaging procedure to check your bone density and examine your skull. We will also need to conduct a plain x-ray of your skull to check for any fractures. Additionally, we may need to perform a complete physical skin exam to check for fractures. Depending on the severity of the fracture, we may also need to do other diagnostic procedures such as a plain x‐ray of your head, a CT scan of your head (MRI), or a plain x‑ray of your eye to evaluate the extent of the damage.
Perplexity: 3.23
prompt: {'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': "Hi doctor, I have a wound on my jaw that won't heal and my wrist is swelling.", 'output': "Based on your symptoms you might have an infection called Ludwig's angina, which can spread from the jaw to the neck and chest. It can also cause swelling in your wrist if left untreated."} and perplexity score: 3.2299602031707764
{'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': "Doctor, I have this open wound on my jaw and I've been experiencing facial pain, mouth pain and also some wrist swelling. It?? been bothering me for a few days now. What do you think is the problem?", 'output': 'Based on your symptoms, you seem to have an infection in the wound and surrounding tissue. This is called cellulitis. We need to start you on antibiotics soon to prevent it from spreading further.'}
Welcome to the Medical Chatbot. Type 'exit' or 'quit' to end the conversation.

Generated Response: Based on your symptoms, it sounds like you may have a severe form of osteoporosis. To confirm the diagnosis, we need to perform a radiographic imaging procedure to check for any fractures. We will also need to conduct a plain x-ray to check for damage. We will need to perform other diagnostic procedures such as a plain x skin exam, a plain x x skin exam to check for discoloration, and a plain x surface skin exam to assess the extent of the damage.
Perplexity: 3.71
prompt: {'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': "Doctor, I have this open wound on my jaw and I've been experiencing facial pain, mouth pain and also some wrist swelling. It?? been bothering me for a few days now. What do you think is the problem?", 'output': 'Based on your symptoms, you seem to have an infection in the wound and surrounding tissue. This is called cellulitis. We need to start you on antibiotics soon to prevent it from spreading further.'} and perplexity score: 3.705753803253174
{'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': "Doctor, I've been experiencing fatigue, sharp abdominal pain, fever and nausea. What's wrong with me?", 'output': 'You might have a white blood cell disease. We need to run some tests to confirm the diagnosis.'}
Welcome to the Medical Chatbot. Type 'exit' or 'quit' to end the conversation.

Generated Response: You may have a condition called hypothyroidism. Hypothyroidism is a condition in which your thyroid gland is unable to produce enough thyroid hormone to help you maintain healthy weight. In order to treat hypothyroidistosis, we will need to perform several medical tests. We will need to start with a complete blood count (CBC), glucose measurement (Glucose measurement), electrolytes panel (Etochemistry panel), lipid panel (Metabolism panel), lipid therapy (Lactate panel), lipid replacement therapy (Methylcellulose panel), lipid transplant (Plasma cell transplant), and kidney function test (Kidney function test). Additionally, we may need to conduct a urinalysis to check for any abnormalities in your kidneys.
Perplexity: 2.89
prompt: {'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': "Doctor, I've been experiencing fatigue, sharp abdominal pain, fever and nausea. What's wrong with me?", 'output': 'You might have a white blood cell disease. We need to run some tests to confirm the diagnosis.'} and perplexity score: 2.885409355163574
{'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': 'Doctor, I have been having a fever and feeling nauseous for the past few days. What could be wrong with me?', 'output': "Based on your symptoms, it's possible that you have white blood cell disease, which can cause fever and nausea. We will need to run some tests to confirm this diagnosis."}  
Welcome to the Medical Chatbot. Type 'exit' or 'quit' to end the conversation.

Generated Response: Based on your symptoms, it seems like you may have a fever. To confirm this, we will need to perform a complete physical skin exam, hematologic tests, and a lipid panel. We will also need to conduct a complete blood count (CBC), a complete blood test, and a urinalysis. Additionally, we will conduct a plain x-ray to check for any abnormalities in your blood vessels.
Perplexity: 3.14
prompt: {'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': 'Doctor, I have been having a fever and feeling nauseous for the past few days. What could be wrong with me?', 'output': "Based on your symptoms, it's possible that you have white blood cell disease, which can cause fever and nausea. We will need to run some tests to confirm this diagnosis."} and perplexity score: 3.144683361053467
{'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': "Doctor, I've been having heartburn, fluid retention, and nausea. What could be causing these symptoms?", 'output': "Based on your symptoms, you may be experiencing complications from a white blood cell disease. It's important that we run some tests to confirm the diagnosis and start treatment as soon as possible."}
Welcome to the Medical Chatbot. Type 'exit' or 'quit' to end the conversation.

Generated Response: In order to diagnose heartburn, we need to perform a complete physical skin exam, hematologic tests, hematology tests, and hematologic test, which includes a complete blood count (CBC), lipid panel, lipid panel, and lipid panel measurement. We will also need to conduct other diagnostic procedures such as an interview, evaluation, and consultation to determine if you have any other medical conditions.
Perplexity: 3.22
prompt: {'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': "Doctor, I've been having heartburn, fluid retention, and nausea. What could be causing these symptoms?", 'output': "Based on your symptoms, you may be experiencing complications from a white blood cell disease. It's important that we run some tests to confirm the diagnosis and start treatment as soon as possible."} and perplexity score: 3.2202792167663574
{'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': "Hi doctor, I've been experiencing sharp abdominal pain, fluid retention, mouth pain, and vomiting. Do you have any idea what could be causing these symptoms?", 'output': 'Yes, based on your symptoms, it is possible that you have a white blood cell disease. We will need some more tests to confirm this diagnosis.'}
Welcome to the Medical Chatbot. Type 'exit' or 'quit' to end the conversation.

Generated Response: Based on your symptoms, it sounds like you may have a bacterial infection. To confirm the diagnosis, we need to perform a complete physical skin exam and evaluation. We will also need to do a urinalysis to check for any abnormalities in your urinary tract. Additionally, we will need to conduct a complete physical physical skin exam to rule out any other causes of your symptoms.
Perplexity: 3.05
prompt: {'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': "Hi doctor, I've been experiencing sharp abdominal pain, fluid retention, mouth pain, and vomiting. Do you have any idea what could be causing these symptoms?", 'output': 'Yes, based on your symptoms, it is possible that you have a white blood cell disease. We will need some more tests to confirm this diagnosis.'} and perplexity score: 3.053339958190918
{'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': "Doctor, I've been experiencing chills, sharp abdominal pain, fluid retention, mouth pain, and diarrhea. What's wrong with me?", 'output': "Based on your symptoms, it's possible that you have a white blood cell disease. I recommend we run some tests to confirm the diagnosis and discuss treatment options."}
Welcome to the Medical Chatbot. Type 'exit' or 'quit' to end the conversation.

Generated Response: Based on your symptoms, it's possible that you have chills. To confirm the diagnosis, we'll need to perform a complete physical skin exam and hematologic tests to check for any abnormalities. We'll also need to do a Hematologic test to check for electrolytes levels and a Lipid panel to check for fatty acids. Additionally, we'll also need a Radiographic imaging procedure to check for other conditions such as kidney function, diabetes, and other health issues.
Perplexity: 3.61
prompt: {'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': "Doctor, I've been experiencing chills, sharp abdominal pain, fluid retention, mouth pain, and diarrhea. What's wrong with me?", 'output': "Based on your symptoms, it's possible that you have a white blood cell disease. I recommend we run some tests to confirm the diagnosis and discuss treatment options."} and perplexity score: 3.612651824951172
{'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': "Doctor, I have been experiencing foot pain for a few weeks now. It's a sharp, stabbing pain in my toes, especially when I walk or stand for a while.", 'output': 'Based on your symptoms, it sounds like you may have Kaposi sarcoma. Have you noticed any unusual skin lesions or discolored patches anywhere on your body?'}
Welcome to the Medical Chatbot. Type 'exit' or 'quit' to end the conversation.

Generated Response: Based on your symptoms, it seems like you may be suffering from a degenerative foot disease. To confirm this, we will need to perform a radiographic imaging procedure to check for any signs of infection. We will also need to conduct a complete physical skin exam to check for discoloration and other abnormalities. Additionally, we will also need a urinalysis to check for kidney function. Finally, we will conduct a biopsy to check for infection.
Perplexity: 3.76
prompt: {'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': "Doctor, I have been experiencing foot pain for a few weeks now. It's a sharp, stabbing pain in my toes, especially when I walk or stand for a while.", 'output': 'Based on your symptoms, it sounds like you may have Kaposi sarcoma. Have you noticed any unusual skin lesions or discolored patches anywhere on your body?'} and perplexity score: 3.760802745819092
{'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': 'Doctor, I have been experiencing excessive growth, sharp abdominal pain, depression, and foot or toe pain. Do you know what could be causing these symptoms?', 'output': 'I am sorry to inform you that all of your symptoms are indicative of Kaposi sarcoma.'}
Welcome to the Medical Chatbot. Type 'exit' or 'quit' to end the conversation.

Generated Response: Yes, it is possible that you may have an osteoarthritis of the foot. To confirm this, we will need to perform a radiographic imaging procedure to measure your foot. We will also need to do a complete physical skin exam to check for any fractures or other complications. Additionally, we will also need a urinalysis to check for infection. In addition, we will conduct a radiographic computed tomography (CT scan) to see if any of the underlying tissue is affected.
Perplexity: 3.37
prompt: {'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': 'Doctor, I have been experiencing excessive growth, sharp abdominal pain, depression, and foot or toe pain. Do you know what could be causing these symptoms?', 'output': 'I am sorry to inform you that all of your symptoms are indicative of Kaposi sarcoma.'} and perplexity score: 3.3667304515838623
{'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': "Hi doctor, I've been experiencing leg pain and I've been losing weight recently. Do you have any idea what could be causing these symptoms?", 'output': 'Yes, based on your symptoms it could be Kaposi sarcoma. Have you noticed any unusual skin lesions or discoloration on your body?'}
Welcome to the Medical Chatbot. Type 'exit' or 'quit' to end the conversation.

Generated Response: Based on your symptoms, it sounds like you might have a degenerative disease. To confirm the diagnosis, we will need to conduct a complete physical skin exam and examination of your body. We will also need to do a biopsy of your pelvis and ophthalmic examination of your eye. Additionally, we may need to perform a corneal transplant or excision of the eyelid to determine if there is a tumor in your eye.
Perplexity: 3.36
prompt: {'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': "Hi doctor, I've been experiencing leg pain and I've been losing weight recently. Do you have any idea what could be causing these symptoms?", 'output': 'Yes, based on your symptoms it could be Kaposi sarcoma. Have you noticed any unusual skin lesions or discoloration on your body?'} and perplexity score: 3.36029052734375
{'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': "Hi doctor, I have been experiencing excessive growth along with low back weakness. I also noticed a skin lesion, and I've been having a cough. What do you think could be the problem?", 'output': 'Based on your symptoms, it seems like you might have Kaposi sarcoma, which is a type of cancer that causes abnormal tissue growth and skin lesions. We need to do some tests to confirm the diagnosis and start appropriate treatment.'}
Welcome to the Medical Chatbot. Type 'exit' or 'quit' to end the conversation.

Generated Response: Based on your symptoms, it sounds like you may have a cough. To confirm the diagnosis, we will need to perform a complete physical skin exam and a urinalysis. We will also need to conduct a radiographic imaging procedure to check for any other possible infections. Additionally, we may need to do a urinal scan to check for infection. Lastly, we may require a complete blood count (CBC) to check for kidney function.
Perplexity: 3.38
prompt: {'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': "Hi doctor, I have been experiencing excessive growth along with low back weakness. I also noticed a skin lesion, and I've been having a cough. What do you think could be the problem?", 'output': 'Based on your symptoms, it seems like you might have Kaposi sarcoma, which is a type of cancer that causes abnormal tissue growth and skin lesions. We need to do some tests to confirm the diagnosis and start appropriate treatment.'} and perplexity score: 3.384087085723877
{'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': 'Doctor, I have been feeling a weakness in my low back for the past few days. Do you know what could be causing this?', 'output': 'Based on your symptoms, it is possible that you have Kaposi sarcoma, which is a type of cancer that affects the skin and other tissues. We should conduct further tests to confirm the diagnosis.'}
Welcome to the Medical Chatbot. Type 'exit' or 'quit' to end the conversation.

Generated Response: Based on your symptoms, we will need to perform some medical tests to determine the cause of this condition. We will need to conduct a radiographic imaging procedure, hematologic tests, hematology tests, and hematologic test, as well as a complete physical skin exam. Additionally, we may need to conduct other diagnostic procedures such as a urinalysis, hematological test, and a complete blood count (CBC).
Perplexity: 2.92
prompt: {'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': 'Doctor, I have been feeling a weakness in my low back for the past few days. Do you know what could be causing this?', 'output': 'Based on your symptoms, it is possible that you have Kaposi sarcoma, which is a type of cancer that affects the skin and other tissues. We should conduct further tests to confirm the diagnosis.'} and perplexity score: 2.9151575565338135
{'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': "Doctor, I've been having a lot of pain and soreness in my breast, and I feel like my legs have been bothering me a lot lately. They feel weak and I get cramps and spasms. ", 'output': "Based on your symptoms, it's possible that you're experiencing some issues with spondylolisthesis. This is a condition where one of the vertebrae in your spine slips out of place, which can put pressure on nerves and cause pain or weakness in your legs. We'll need to do some testing to confirm the diagnosis, but there are treatment options available to help manage your symptoms."}
Welcome to the Medical Chatbot. Type 'exit' or 'quit' to end the conversation.

Generated Response: It sounds like you may have a condition called Muscular Dystrophy. We'll need to do some tests to confirm this diagnosis. We'll start with a radiographic imaging procedure to see if you have any abnormalities in your breast tissue. We'll also need to do a complete physical skin exam to check for any signs of infection. Additionally, we'll need to perform a complete physical therapy including an interview, consultation, and evaluation. We'll run a complete blood count (CBC) and glucose measurement (Glucose level) to determine if you have diabetes. Lastly, we'll conduct a lipid panel to check for fatty acids and cholesterol levels.
Perplexity: 3.46
prompt: {'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': "Doctor, I've been having a lot of pain and soreness in my breast, and I feel like my legs have been bothering me a lot lately. They feel weak and I get cramps and spasms. ", 'output': "Based on your symptoms, it's possible that you're experiencing some issues with spondylolisthesis. This is a condition where one of the vertebrae in your spine slips out of place, which can put pressure on nerves and cause pain or weakness in your legs. We'll need to do some testing to confirm the diagnosis, but there are treatment options available to help manage your symptoms."} and perplexity score: 3.4617412090301514
{'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': "Doctor, I have been experiencing low back pain and neck pain for a few weeks now. Can you please help understand what's causing this?", 'output': "Based on your symptoms, it seems like you could be suffering from a condition called spondylolisthesis. It's a spinal disorder that results in vertebrae slipping out of place and can cause persistent back pain and stiffness."}
Welcome to the Medical Chatbot. Type 'exit' or 'quit' to end the conversation.

Generated Response: Yes, we need to start with a radiographic imaging procedure to examine your neck. We will also need to perform a complete physical skin exam, including a cornea examination, which will help us determine the cause of your pain. Additionally, we may need to conduct a radiographic computed tomography (CT) to check for any fractures or other injuries. Additionally, a plain x-ray computed tomography or X-ray computed computed tomography may be necessary to confirm the diagnosis.
Perplexity: 3.72
prompt: {'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': "Doctor, I have been experiencing low back pain and neck pain for a few weeks now. Can you please help understand what's causing this?", 'output': "Based on your symptoms, it seems like you could be suffering from a condition called spondylolisthesis. It's a spinal disorder that results in vertebrae slipping out of place and can cause persistent back pain and stiffness."} and perplexity score: 3.724975824356079
{'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': 'Doctor, I am experiencing leg pain, leg cramps, weakness, and difficulty in moving. Also, I have been having severe back pain for a while now.', 'output': 'Based on your symptoms, it seems like you are suffering from Spondylolisthesis. It is a condition where one of the vertebrae in your spine slips out of its correct position and causes pressure on the surrounding nerves, resulting in the symptoms you described.'}
Welcome to the Medical Chatbot. Type 'exit' or 'quit' to end the conversation.

Generated Response: Based on your symptoms, it is possible that you may have osteoarthritis. To confirm this, we will need to perform a radiographic imaging procedure, including a plain x-ray, or hematologic test, which includes a complete physical skin exam, hematologic tests, and hematologic testing, such as a complete blood count (CBC), and a complete kidney function test. Additionally, we may need to do a kidney function test, kidney function panel, kidney function tests, and kidney function assessment.
Perplexity: 2.63
prompt: {'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': 'Doctor, I am experiencing leg pain, leg cramps, weakness, and difficulty in moving. Also, I have been having severe back pain for a while now.', 'output': 'Based on your symptoms, it seems like you are suffering from Spondylolisthesis. It is a condition where one of the vertebrae in your spine slips out of its correct position and causes pressure on the surrounding nerves, resulting in the symptoms you described.'} and perplexity score: 2.634500741958618
{'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': 'Doctor, I have been experiencing back pain, loss of sensation, and neck pain. What could be the problem?', 'output': 'These symptoms may suggest that you have Spondylolisthesis.'}
Welcome to the Medical Chatbot. Type 'exit' or 'quit' to end the conversation.

Generated Response: In order to diagnose the underlying cause of your back pain, we will need to perform a series of medical tests including a complete physical skin exam, examination of the skin, and evaluation of the underlying tissues. We will also need to conduct other diagnostic procedures such as hematologic tests, hematologic test, and kidney function tests. Additionally, we may need to conduct a radiographic imaging procedure to examine your kidneys and other organs.
Perplexity: 2.98
prompt: {'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': 'Doctor, I have been experiencing back pain, loss of sensation, and neck pain. What could be the problem?', 'output': 'These symptoms may suggest that you have Spondylolisthesis.'} and perplexity score: 2.9762356281280518
{'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': 'Doctor, I have been experiencing leg cramps, paresthesia, and lower body pain. Do you know what might be causing this?', 'output': 'Yes, it sounds like you may have spondylolisthesis, which is a condition where one of the bones in your spine slips out of place and puts pressure on the nerves. These symptoms are commonly associated with this condition.'}
Welcome to the Medical Chatbot. Type 'exit' or 'quit' to end the conversation.

Generated Response: Yes, we will need to perform some medical tests to determine the cause of your condition. We will start with a complete blood count (CBC) and a urinalysis to check for any abnormal fluid levels. Additionally, we will conduct a urinal culture to confirm the presence of kidney and bladder cells in your urine. Lastly, we will also need to do a biopsy to rule out any other causes of the condition.
Perplexity: 3.86
prompt: {'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': 'Doctor, I have been experiencing leg cramps, paresthesia, and lower body pain. Do you know what might be causing this?', 'output': 'Yes, it sounds like you may have spondylolisthesis, which is a condition where one of the bones in your spine slips out of place and puts pressure on the nerves. These symptoms are commonly associated with this condition.'} and perplexity score: 3.8569304943084717
{'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': 'Doctor, I have been experiencing pain in my eye, fluid retention, diminished vision, and regurgitation. What might be the problem?', 'output': "Based on the symptoms you're describing, you may have pseudotumor cerebri, which is also known as idiopathic intracranial hypertension."}
Welcome to the Medical Chatbot. Type 'exit' or 'quit' to end the conversation.

Generated Response: In order to diagnose and treat your condition, we will need to perform a series of tests including a complete blood count (CBC), a plain x-ray computed tomography (CT scan), and a plain x image computed tomography to check for any abnormalities in your retina. We will also need to conduct a lipid panel to monitor your cholesterol levels and glucose levels. Additionally, we may need to conduct other diagnostic procedures such as a complete blood measurement (CBC), lipid panel analysis (CBC), and evaluation of your cholesterol level.
Perplexity: 3.32
prompt: {'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': 'Doctor, I have been experiencing pain in my eye, fluid retention, diminished vision, and regurgitation. What might be the problem?', 'output': "Based on the symptoms you're describing, you may have pseudotumor cerebri, which is also known as idiopathic intracranial hypertension."} and perplexity score: 3.3182873725891113
{'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': 'Doctor, I have been experiencing diminished vision, fluid retention, headaches, abnormal involuntary movements, and spots or clouds in my vision. Do you know what could be causing this?', 'output': 'Based on your symptoms, you may be suffering from a condition called pseudotumor cerebri. It occurs when there is increased pressure inside the skull, which can lead to vision problems, headaches, and other symptoms. I recommend further testing to confirm this diagnosis.'}
Welcome to the Medical Chatbot. Type 'exit' or 'quit' to end the conversation.

Generated Response: Based on your symptoms, it is possible that you may have a rare form of ophthalmic syndrome. We will need to conduct a complete physical skin exam, including a cornea examination, a corneal transplant, and a plain x-ray to determine the cause of your condition. We will also need to do a corneoscopy, a cornea measurement, and a biopsy to confirm the diagnosis. Additionally, we may need to perform a corneogram or a cataract or other diagnostic procedure to check for any underlying conditions.
Perplexity: 3.46
prompt: {'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': 'Doctor, I have been experiencing diminished vision, fluid retention, headaches, abnormal involuntary movements, and spots or clouds in my vision. Do you know what could be causing this?', 'output': 'Based on your symptoms, you may be suffering from a condition called pseudotumor cerebri. It occurs when there is increased pressure inside the skull, which can lead to vision problems, headaches, and other symptoms. I recommend further testing to confirm this diagnosis.'} and perplexity score: 3.460116147994995
{'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': "Hi doctor, I've been experiencing some strange symptoms lately. I've been seeing spots or clouds in my vision, feeling dizzy, and experiencing pain in my eye. Can you tell me what might be causing this?", 'output': "Based on your symptoms, it's possible that you may be suffering from Pseudotumor cerebri. This is a condition where there is increased pressure inside the skull, which can lead to vision problems, dizziness, and eye pain. We will need to conduct more tests to confirm the diagnosis."}  
Welcome to the Medical Chatbot. Type 'exit' or 'quit' to end the conversation.

Generated Response: Based on your symptoms, we will need to perform a few medical tests to confirm the diagnosis. We will need to start with a complete blood count (CBC), hematologic tests (blood test), a complete blood work (CBC), and hematologic examination (blood test). We may also need to conduct a urinalysis to check for any fluid or electrolytes in your urine. We may also require a plain x-ray to check for abnormalities in your kidneys. Additionally, we may need to do a complete blood test (CBC) to rule out any other medical conditions that may be causing the condition. Lastly, we may also need a urinal culture to check for signs of kidney function.
Perplexity: 3.33
prompt: {'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': "Hi doctor, I've been experiencing some strange symptoms lately. I've been seeing spots or clouds in my vision, feeling dizzy, and experiencing pain in my eye. Can you tell me what might be causing this?", 'output': "Based on your symptoms, it's possible that you may be suffering from Pseudotumor cerebri. This is a condition where there is increased pressure inside the skull, which can lead to vision problems, dizziness, and eye pain. We will need to conduct more tests to confirm the diagnosis."} and perplexity score: 3.3320260047912598
{'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': "Doctor, I've been experiencing pain in my eye along with dizziness and slurring of words. Also, I've noticed spots or clouds in my vision lately. Can you tell me what's happening to me?", 'output': 'These symptoms could be due to pseudotumor cerebri, a condition in which the pressure inside your skull increases for no apparent reason.'}
Welcome to the Medical Chatbot. Type 'exit' or 'quit' to end the conversation.

Generated Response: Based on your symptoms, we need to perform a few medical tests. We'll start with a complete blood count (CBC) to check for any abnormalities in your eyes. Additionally, we'll conduct a magnetic resonance imaging (MRI) to check your brain for any changes in your brain activity. Finally, we'll also conduct a complete physical skin exam to check for anything abnormal in your skin.
Perplexity: 3.25
prompt: {'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': "Doctor, I've been experiencing pain in my eye along with dizziness and slurring of words. Also, I've noticed spots or clouds in my vision lately. Can you tell me what's happening to me?", 'output': 'These symptoms could be due to pseudotumor cerebri, a condition in which the pressure inside your skull increases for no apparent reason.'} and perplexity score: 3.249298572540283
{'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': 'Hi doctor, I have been experiencing spots or clouds in my vision, involuntary movements, skin swelling, and fluid retention. What could be the problem?', 'output': 'These symptoms suggest that you may have pseudotumor cerebri.'}
Welcome to the Medical Chatbot. Type 'exit' or 'quit' to end the conversation.

Generated Response: Based on your symptoms, we need to perform some medical tests to confirm the diagnosis. We will start with a complete blood count (CBC) and hematologic tests (blood test) to check for any abnormalities in your blood. Additionally, we may need to conduct a plain x-ray computed tomography (Scan ct) to check your blood clotting levels. We may also need to do a complete physical skin exam (X-ray) to check any other abnormalities in your skin. Finally, we may require a urinalysis to check for anything that might be causing the condition.
Perplexity: 2.75
prompt: {'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': 'Hi doctor, I have been experiencing spots or clouds in my vision, involuntary movements, skin swelling, and fluid retention. What could be the problem?', 'output': 'These symptoms suggest that you may have pseudotumor cerebri.'} and perplexity score: 2.7475786209106445
{'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': "Doctor, I'm having trouble seeing properly and my eyes burn and sting a lot. Also, there's excessive tearing. What could be the problem?", 'output': "From the symptoms you've described, it seems like you are suffering from Conjunctivitis caused by a virus."}
Welcome to the Medical Chatbot. Type 'exit' or 'quit' to end the conversation.

Generated Response: We'll need to run some tests to confirm your diagnosis. We'll start with a complete blood count (CBC) and a plain x-ray of your eye. We'll also need to conduct a cornea measurement to check for any abnormalities in your cornea. Additionally, we'll need to do a radiographic imaging procedure to see if there's any inflammation or damage in your corneal tissue. Lastly, we'll perform a plain x‐ray of your cataract, a corneal transplant, and a corneoscopy to check for other complications.
Perplexity: 3.39
prompt: {'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': "Doctor, I'm having trouble seeing properly and my eyes burn and sting a lot. Also, there's excessive tearing. What could be the problem?", 'output': "From the symptoms you've described, it seems like you are suffering from Conjunctivitis caused by a virus."} and perplexity score: 3.393413782119751
{'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': "Doctor, I have a cough and a strange white discharge coming out of my eye. What's wrong with me?", 'output': 'You have viral conjunctivitis which means you have an infection that causes inflammation in your eye. You may also experience redness and itching along with the other symptoms. Patient: Oh, okay. What treatment do you recommend for this?'}
Welcome to the Medical Chatbot. Type 'exit' or 'quit' to end the conversation.

Generated Response: Based on your symptoms, it is possible that you have a cough. To confirm the diagnosis, we will need to perform a complete blood count (CBC) and a Hemoglobin A1c measurement (Hemoglobin A1C measurement). We will also need to conduct a urinalysis to check for any fluid or electrolytes in your system. Additionally, we will run a plain x-ray computed tomography (Scan ct) to check for signs of infection. Lastly, we will conduct a lipid panel to assess your cholesterol levels.
Perplexity: 2.89
prompt: {'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': "Doctor, I have a cough and a strange white discharge coming out of my eye. What's wrong with me?", 'output': 'You have viral conjunctivitis which means you have an infection that causes inflammation in your eye. You may also experience redness and itching along with the other symptoms. Patient: Oh, okay. What treatment do you recommend for this?'} and perplexity score: 2.8942458629608154
{'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': 'Doctor, I have been experiencing nasal congestion, my eyes burn and sting, and I have been coughing a lot. What could be wrong with me?', 'output': 'Based on the symptoms you have described, it seems like you may have conjunctivitis due to a virus.'}
Welcome to the Medical Chatbot. Type 'exit' or 'quit' to end the conversation.

Generated Response: In order to diagnose nasal congestion, we will need to perform a series of tests including a complete blood count (CBC), hematologic tests (blood test), urinalysis (CBC), and a complete physical skin exam (pulmonary function test). Additionally, we may need to conduct a plain x-ray to check for any abnormalities in your lung tissue.
Perplexity: 3.07
prompt: {'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': 'Doctor, I have been experiencing nasal congestion, my eyes burn and sting, and I have been coughing a lot. What could be wrong with me?', 'output': 'Based on the symptoms you have described, it seems like you may have conjunctivitis due to a virus.'} and perplexity score: 3.0718834400177
{'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': "Doctor, I have a swollen eye and white discharge coming out of it. I also have nasal congestion and my eyes burn and sting. I also have a fever. What's wrong with me?", 'output': 'You have conjunctivitis due to a virus.'}
Welcome to the Medical Chatbot. Type 'exit' or 'quit' to end the conversation.

Generated Response: You may have a fever that is due to a combination of allergies. We will need to run some tests to determine if you have asthma. We will also need to conduct a radiographic imaging procedure to check your face and determine if there is any inflammation in your skin. Additionally, we may need to do a complete blood count (CBC) to check for any blood clotting issues.
Perplexity: 3.69
prompt: {'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': "Doctor, I have a swollen eye and white discharge coming out of it. I also have nasal congestion and my eyes burn and sting. I also have a fever. What's wrong with me?", 'output': 'You have conjunctivitis due to a virus.'} and perplexity score: 3.6920924186706543
{'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': "Hi doctor, I'm experiencing some vision problems and my eyes are really red. Additionally, I noticed some white discharge coming out of my eye. What could be the problem?", 'output': 'Based on your symptoms, it sounds like you have conjunctivitis due to a virus. This is also commonly referred to as pink eye.'}
Welcome to the Medical Chatbot. Type 'exit' or 'quit' to end the conversation.

Generated Response: Based on your symptoms, it sounds like you may have a cornea condition. We will need to perform a complete blood count (CBC) to check for any abnormal blood vessels in your eye. We will also need to do a radiographic imaging procedure to examine your cornea and evaluate your condition. We may also need to conduct a corneal transplant, which can help us determine if there is any underlying condition that might affect your vision.
Perplexity: 3.49
prompt: {'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': "Hi doctor, I'm experiencing some vision problems and my eyes are really red. Additionally, I noticed some white discharge coming out of my eye. What could be the problem?", 'output': 'Based on your symptoms, it sounds like you have conjunctivitis due to a virus. This is also commonly referred to as pink eye.'} and perplexity score: 3.4890122413635254
{'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': 'Doctor, I have an open wound in my nose and I also feel some knee weakness, diminished hearing, facial pain, and neck weakness. What could be the problem?', 'output': 'Based on your symptoms, you may have a complication from a middle ear infection, which is causing nerve damage leading to facial pain, neck weakness, and diminished hearing. The knee weakness could be unrelated.'}
Welcome to the Medical Chatbot. Type 'exit' or 'quit' to end the conversation.

Generated Response: In order to diagnose the condition, we will need to perform several medical tests including a radiographic imaging procedure, a plain x-ray, a plain ophthalmic examination, and a complete physical skin exam. We will also need to conduct other diagnostic procedures such as hematologic tests, hematologic test, kidney function tests, and a lipid panel. Additionally, we may need to perform a Hemoglobin A1c measurement (Hemoglobin A1C level) and a lipid screen.
Perplexity: 2.81
prompt: {'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': 'Doctor, I have an open wound in my nose and I also feel some knee weakness, diminished hearing, facial pain, and neck weakness. What could be the problem?', 'output': 'Based on your symptoms, you may have a complication from a middle ear infection, which is causing nerve damage leading to facial pain, neck weakness, and diminished hearing. The knee weakness could be unrelated.'} and perplexity score: 2.812701940536499
{'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': "Doctor, I have an open wound in my nose and my knee feels weak, and my hearing has also diminished. What's wrong with me?", 'output': 'Based on your symptoms, you may have a staph infection in the wound which could be affecting your knee and causing hearing loss. We need to start treatment immediately.'}
Welcome to the Medical Chatbot. Type 'exit' or 'quit' to end the conversation.

Generated Response: Based on your symptoms, we need to perform some medical tests to determine if you have osteoporosis. We will need to do a radiographic imaging procedure to examine your skull, a plain x-ray (MRI), a plain x x-ray, and a complete physical skin exam to check for any fractures. We may also need to conduct a biopsy to confirm any other conditions that might be causing the fracture. Additionally, we may need to do an examination of your foot, pelvis, and hip to determine whether you have osteoarthritis.
Perplexity: 3.53
prompt: {'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': "Doctor, I have an open wound in my nose and my knee feels weak, and my hearing has also diminished. What's wrong with me?", 'output': 'Based on your symptoms, you may have a staph infection in the wound which could be affecting your knee and causing hearing loss. We need to start treatment immediately.'} and perplexity score: 3.5252442359924316
{'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': "Doctor, I have an open wound on my nose and I'm experiencing hot flashes. What could be the cause of these symptoms?", 'output': "Hot flashes are a common symptom of menopause, but they are not usually associated with an open wound on the nose. Based on your symptoms, you may have an infection in the wound that's causing the feverish feeling."}
Welcome to the Medical Chatbot. Type 'exit' or 'quit' to end the conversation.

Generated Response: In order to confirm the diagnosis, we will need to perform a radiographic imaging procedure, including a plain x-ray, and a complete blood count (CBC). We will also need to conduct a urinalysis and electrolytes panel to check for any fluid buildup. Additionally, we may need to conduct an electrolytes panel and a lipid panel to monitor your blood sugar levels.
Perplexity: 2.80
prompt: {'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': "Doctor, I have an open wound on my nose and I'm experiencing hot flashes. What could be the cause of these symptoms?", 'output': "Hot flashes are a common symptom of menopause, but they are not usually associated with an open wound on the nose. Based on your symptoms, you may have an infection in the wound that's causing the feverish feeling."} and perplexity score: 2.8044378757476807
{'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': 'Doctor, I have a sore in my nose and I keep getting headaches. Also, my nose has been bleeding a lot lately and my face hurts.', 'output': 'It sounds like you have an open wound in your nose. This can cause facial pain, nosebleeds, and headaches. We need to clean and monitor the wound to avoid infections.'}
Welcome to the Medical Chatbot. Type 'exit' or 'quit' to end the conversation.

Generated Response: Okay, I'll start with some tests to see if you have any underlying conditions. We'll need to do a radiographic imaging procedure, a plain x-ray, a plain X-ray, and a plain X chromosome scan to check for any abnormalities. Additionally, we'll need to perform a lipid panel and other diagnostic procedures to check for diabetes, hypertension, and osteoporosis. Finally, we'll conduct a urinalysis to check for kidney function.
Perplexity: 4.06
prompt: {'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': 'Doctor, I have a sore in my nose and I keep getting headaches. Also, my nose has been bleeding a lot lately and my face hurts.', 'output': 'It sounds like you have an open wound in your nose. This can cause facial pain, nosebleeds, and headaches. We need to clean and monitor the wound to avoid infections.'} and perplexity score: 4.0561347007751465
{'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': 'Doctor, I have an open wound on my nose, and I am experiencing hot flashes, facial pain, and diminished hearing. What could be the problem?', 'output': 'It seems like you have an infection in your nose which is causing an open wound and as a result, you are experiencing hot flashes, facial pain, and diminished hearing. I believe you have sinusitis.'}
Welcome to the Medical Chatbot. Type 'exit' or 'quit' to end the conversation.

Generated Response: In order to diagnose and treat this condition, we will need to perform a radiographic imaging procedure to evaluate your face. We will also need to do a plain x-ray to see if there is any inflammation or damage to your nasal tissue. We may also need to conduct a corneal exam to check for any other problems in your nasal cavity. Additionally, we may need to conduct other diagnostic procedures such as a plain x x-ray, a plain x ophthalmic examination, and a complete physical skin exam.
Perplexity: 3.44
prompt: {'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': 'Doctor, I have an open wound on my nose, and I am experiencing hot flashes, facial pain, and diminished hearing. What could be the problem?', 'output': 'It seems like you have an infection in your nose which is causing an open wound and as a result, you are experiencing hot flashes, facial pain, and diminished hearing. I believe you have sinusitis.'} and perplexity score: 3.441157817840576
The perplexity calculated for the test set is: 3.3209633207321168
"""

In [ ]:
# Regex to extract prompts and perplexity scores
import pandas as pd
import re

pattern = r"prompt: (.*?) and perplexity score: ([\d.]+)"

# Extract matches
matches = re.findall(pattern, data, re.DOTALL)

# Create DataFrame
df = pd.DataFrame(matches, columns=["Prompt", "Perplexity"])

# Convert Perplexity column to numeric type
df["Perplexity"] = pd.to_numeric(df["Perplexity"])

# Display the table to the user
import ace_tools as tools; tools.display_dataframe_to_user(name="Prompt and Perplexity Table", dataframe=df)

In [ ]:
def main():
    # Load the fine-tuned model and tokenizer
    model_path = './trained_lora_2k'  
    tokenizer = GPT2Tokenizer.from_pretrained(model_path)
    model = GPT2LMHeadModel.from_pretrained(model_path)

    # Move model to the appropriate device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    model.eval()

    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    if model.config.pad_token_id is None:
        model.config.pad_token_id = model.config.eos_token_id

  
    print("Welcome to the Medical Chatbot. Type 'exit' or 'quit' to end the conversation.")


    instruction = (
        "You are an experienced medical doctor"
        "Provide a detailed and compassionate treatment plan to the patient's description based on the symptoms the patient describes."
    )

    while True:
        # Get user input
        user_input = input("\nPatient: ")
        if user_input.lower() in ['exit', 'quit']:
            print("\nChatbot: Take care! Goodbye.")
            break

        # Prepare the prompt
        prompt = (
            f"{instruction}\n\nPatient's Description: {user_input}\n\nDoctor's Response:"
        )

        # Tokenize the prompt
        inputs = tokenizer(prompt, return_tensors='pt').to(device)

        # Generate a response
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=200,
                do_sample=True,
                temperature=0.8,
                top_p=0.5,
                repetition_penalty=1.01,
                no_repeat_ngram_size=4,
                pad_token_id=tokenizer.eos_token_id,
            )

   
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

   
        response_start = generated_text.find("Doctor's Response:")
        if response_start != -1:
            response = generated_text[response_start + len("Doctor's Response:"):].strip()
        else:
            response = generated_text[len(prompt):].strip()

  
        if response.startswith("Patient's Description:"):
            response = response.split("Patient's Description:")[0].strip()

     
        print(f"\nDoctor: {response}")

if __name__ == "__main__":
    main()
